In [1]:
from config.configuration import engine
from tools import idealista_functions as i_func
import pandas as pd

Conectado a mysql / base de datos: tu_pisito


In [ ]:
mysql_df = pd.read_sql_query(
f"""
SELECT sq_mt_built, 
        n_rooms, 
        n_bathrooms, 
        apartments.latitude, 
        apartments.longitude, 
        is_new_development, 
        is_renewal_needed, 
        has_lift, 
        is_exterior, 
        has_parking, 
        floor, 
        house_type_id,
        value_m2,
        neighborhood_name,
        district_name,
        buy_price
FROM apartments
LEFT JOIN neighborhoods
ON apartments.neighborhood_id = neighborhoods.neighborhood_id
LEFT JOIN districts
ON neighborhoods.district_id = districts.district_id
ORDER BY apartment_id DESC;
""", engine)